In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

TypeError: str expected, not NoneType

In [2]:
import boto3

# Create an STS client
sts_client = boto3.client('sts')

try:
    # Call the get_caller_identity() method
    identity = sts_client.get_caller_identity()
    
    print("✅ Successfully authenticated with AWS.")
    print("---------------------------------------")
    print(f"Account ID: {identity['Account']}")
    print(f"User ID:    {identity['UserId']}")
    print(f"ARN:        {identity['Arn']}")
    
except Exception as e:
    print(f"❌ Error authenticating with AWS: {e}")
    print("Please check your credentials (e.g., run 'aws configure' in your terminal).")

session = boto3.Session()

# Check the profile name boto3 is configured to use
print(f"Using AWS Profile: {session.profile_name}")

✅ Successfully authenticated with AWS.
---------------------------------------
Account ID: 322691663228
User ID:    AIDAUWIPMAV6OANS3KFHN
ARN:        arn:aws:iam::322691663228:user/langsmith-bedrock-user
Using AWS Profile: langsmith-bedrock-user


In [3]:
from langchain_aws import ChatBedrock
llm = ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0", # Or any other model you have access to
    region_name="us-east-1"      # Or your preferred region
)
print(llm)

client=<botocore.client.BedrockRuntime object at 0x71e1dcb41590> bedrock_client=<botocore.client.Bedrock object at 0x71e1dc857790> region_name='us-east-1' model_id='meta.llama3-8b-instruct-v1:0' model_kwargs={}


In [4]:
## Input and get response form LLM

result=llm.invoke("What is generative AI?")

In [5]:
print(result)

content='Generative AI refers to a type of artificial intelligence (AI) that can create new, original content, such as text, images, music, or videos, rather than simply processing or analyzing existing data. Generative AI models use algorithms and machine learning techniques to generate new data that is similar in style, tone, and quality to existing data, but is not necessarily identical to it.\n\nGenerative AI has many applications, including:\n\n1. **Art and design**: Generative AI can create new art, music, and designs, such as generating new images, music tracks, or even entire albums.\n2. **Content creation**: Generative AI can help create new content, such as blog posts, articles, or social media posts, by generating text based on a given topic or style.\n3. **Data augmentation**: Generative AI can be used to augment existing datasets by generating new, synthetic data that is similar to the original data.\n4. **Chatbots and conversational AI**: Generative AI can be used to crea

In [6]:
### Chatprompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer. Provide me answers based on the questions"),
        ("user","{input}")
    ]

)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answers based on the questions'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [7]:
## chain 
chain=prompt|llm

response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

content="Langsmith is a fascinating topic!\n\nLangsmith is an AI-powered language engineering platform that enables developers to build, train, and deploy custom language models for various applications. The platform provides a range of tools and services to help developers create high-quality language models that can be used for natural language processing (NLP) tasks such as text classification, sentiment analysis, language translation, and more.\n\nSome of the key features of Langsmith include:\n\n1. **Model training**: Langsmith allows developers to train custom language models using their own datasets, which can be used for specific applications.\n2. **Model deployment**: Once trained, Langsmith enables developers to deploy their models in various environments, including cloud, on-premises, and mobile devices.\n3. **Model management**: Langsmith provides a centralized platform for managing multiple language models, making it easy to track performance, update models, and scale depl

In [8]:
type(response)

langchain_core.messages.ai.AIMessage

In [9]:
## stroutput Parser

from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
chain=prompt|llm|output_parser

response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

Langsmith is an open-source, cloud-based platform that enables data scientists and engineers to build, deploy, and manage machine learning models at scale. It provides a range of tools and services to streamline the machine learning workflow, from data preparation to model deployment.

Some of the key features of Langsmith include:

1. **Model Management**: Langsmith allows users to manage their machine learning models in a centralized repository, making it easy to track versions, collaborate with team members, and deploy models to production.
2. **Automated Model Training**: Langsmith provides automated model training capabilities, allowing users to train models on large datasets without manual intervention.
3. **Model Serving**: Langsmith provides a scalable model serving platform that can handle high-traffic and high-volume requests, making it suitable for production environments.
4. **Data Preparation**: Langsmith includes data preparation tools that enable users to preprocess and 